# Tokenizer

(samples modified from:  https://huggingface.co/docs/transformers/main/en/preprocessing)

In [1]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")

In [2]:
# see: https://huggingface.co/docs/transformers/main/en/preprocessing
encoded_input = tokenizer("Do not meddle in the affairs of wizards, for they are subtle and quick to anger.")
print(encoded_input['input_ids'])
print(encoded_input['token_type_ids'])
print(encoded_input['attention_mask'])

[101, 2091, 1136, 1143, 13002, 1107, 1103, 5707, 1104, 16678, 1116, 117, 1111, 1152, 1132, 11515, 1105, 3613, 1106, 4470, 119, 102]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


# Vacabulary and Raw Tokens

In [3]:
from transformers import GPT2Tokenizer # Initialize the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
text = "We like Transformers in 2024!"

# Tokenize the text
token_ids = tokenizer.encode(text, add_special_tokens=True) # Output the token IDs
print("Token IDs:", token_ids)

# Convert token IDs back to raw tokens and output them
raw_tokens = [tokenizer.decode([token_id]) for token_id in token_ids]
print("Raw tokens:", raw_tokens)

Token IDs: [1135, 588, 39185, 287, 48609, 0]
Raw tokens: ['We', ' like', ' Transformers', ' in', ' 2024', '!']


# Tokens <> Embeddings

In [4]:
from transformers import BertTokenizer, BertModel
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased') # Load pre-trained model tokenizer
model = BertModel.from_pretrained('bert-base-uncased') # Load pre-trained model
text = "We like Transformers in 2024!" # Text to be tokenized
input_ids = tokenizer.encode(text, add_special_tokens=True) 
print("Token IDs:", input_ids) # Output the token IDs

# Convert token IDs back to raw tokens and output them
raw_tokens = [tokenizer.decode([token_id]) for token_id in input_ids]
print("Raw tokens:", raw_tokens)

# Convert list of IDs to a tensor
input_ids_tensor = torch.tensor([input_ids])

# Pass the input through the model
with torch.no_grad():
    outputs = model(input_ids_tensor)

# Extract the embeddings
embeddings = outputs.last_hidden_state

# Print the embeddings
print("Embeddings: ", embeddings)

Token IDs: [101, 2057, 2066, 19081, 1999, 16798, 2549, 999, 102]
Raw tokens: ['[CLS]', 'we', 'like', 'transformers', 'in', '202', '##4', '!', '[SEP]']
Embeddings:  tensor([[[ 0.2137, -0.0775,  0.3104,  ..., -0.3053,  0.6714,  0.1316],
         [ 0.5586, -0.0104, -0.3371,  ..., -0.2438,  1.4860, -0.1893],
         [ 0.4292,  0.2395,  1.1432,  ..., -0.2679,  0.2587, -0.3247],
         ...,
         [ 0.0826, -0.0128,  0.7097,  ..., -0.5649,  0.1436,  0.4176],
         [ 0.0555, -0.2149,  0.0038,  ...,  0.6719,  0.2073, -0.4478],
         [ 0.7300,  0.0909,  0.2141,  ...,  0.1088, -0.4650, -0.4151]]])


### Dimension der Embeddings.
Für jedes der Token (9) gibt es einen Tensor (einen Vektor) mit 768 Parametern

In [5]:
embeddings.shape

torch.Size([1, 9, 768])

### Darstellung eines Beispiel-Tensors:

In [6]:
embeddings[0][1]

tensor([ 5.5860e-01, -1.0373e-02, -3.3707e-01,  2.8159e-01,  8.0756e-03,
        -1.5892e-01,  7.8681e-02,  7.3727e-01,  6.6016e-01, -4.6414e-01,
        -2.8798e-01, -1.8046e-01,  6.8951e-02,  3.2863e-01, -6.2951e-01,
        -2.7793e-01,  2.0718e-01,  3.4607e-01,  3.7537e-01,  1.7058e-01,
        -3.6978e-01, -5.1181e-02, -1.7712e-01,  3.1384e-01,  3.4984e-01,
        -1.2083e-01, -8.5585e-02,  2.0418e-01,  4.4767e-01, -1.6739e-01,
        -7.6793e-03,  1.3199e-01, -1.3449e-01,  2.6002e-01, -5.7250e-01,
        -3.4680e-01, -5.1936e-01,  6.4652e-01,  4.2504e-02,  2.2167e-01,
         3.1272e-01, -5.2754e-01,  3.5649e-01, -2.1290e-01,  1.1017e-01,
        -2.9269e-01,  4.7714e-03, -1.7892e-01, -1.5575e-01, -8.6869e-01,
        -5.7148e-02,  1.9105e-01, -2.1960e-01,  5.4789e-01, -2.9957e-01,
         1.3120e-01, -2.6315e-01,  5.3071e-01,  3.3026e-01,  2.7262e-01,
        -2.7460e-01,  2.0843e-01, -9.0856e-02, -6.0086e-01, -2.3924e-01,
         3.0926e-01, -4.3366e-01, -4.1676e-01,  5.5

# Handling multiple Sentences

In [7]:
batch_sentences = [
    "But what about second breakfast?",
    "Don't think he knows about second breakfast, Pip.",
    "What about elevensies?",
]
encoded_inputs = tokenizer(batch_sentences)

# Wir definieren eine kleine Hilfsfunktion, um die Parameter ausgeben zu können:
def outp (inputs):
    for name in ['input_ids', 'token_type_ids', 'attention_mask']:
        if name in inputs:
            print(f"\n{name}")
            for i in range (len (inputs[name])):
                print(inputs[name][i])

outp(encoded_inputs)


input_ids
[101, 2021, 2054, 2055, 2117, 6350, 1029, 102]
[101, 2123, 1005, 1056, 2228, 2002, 4282, 2055, 2117, 6350, 1010, 28315, 1012, 102]
[101, 2054, 2055, 5408, 14625, 1029, 102]

token_type_ids
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]

attention_mask
[1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1]


# Padding & Truncation

In [8]:
encoded_inputs = tokenizer(batch_sentences, padding=True, truncation=True)

In [9]:
outp(encoded_inputs)


input_ids
[101, 2021, 2054, 2055, 2117, 6350, 1029, 102, 0, 0, 0, 0, 0, 0]
[101, 2123, 1005, 1056, 2228, 2002, 4282, 2055, 2117, 6350, 1010, 28315, 1012, 102]
[101, 2054, 2055, 5408, 14625, 1029, 102, 0, 0, 0, 0, 0, 0, 0]

token_type_ids
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

attention_mask
[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]


# Word based tokenizer

In [10]:
text = 'But what about second breakfast?'
tokenizer.tokenize(text)

['but', 'what', 'about', 'second', 'breakfast', '?']

# Sub-Word based tokenizer

In [11]:
from transformers import XLNetTokenizer

tokenizer = XLNetTokenizer.from_pretrained("xlnet/xlnet-base-cased")
output = tokenizer.tokenize("Don't you love 🤗 Transformers? We sure do.")
for i in range (len (output)):
    print(output[i], end=' | ')

▁Don | ' | t | ▁you | ▁love | ▁ | 🤗 | ▁ | Transform | ers | ? | ▁We | ▁sure | ▁do | . | 

- Transformers wurde in zwei Sub-Words aufgeteilt
- Satzzeichen habe eigene Tokens

# Your own Tokenizer

## Einen Textcorpus laden
[Credits to huggingface: https://github.com/huggingface/notebooks/blob/main/examples/tokenizer_training.ipynb]

## Ein Dataset von huggingface laden:
Wir laden zunächst das "Wikitext" Dataset von huggingface. Darin enthalten sind Beispieltexte von Wikipedia.

In [12]:
from datasets import load_dataset

In [13]:
dataset = load_dataset("wikitext", name="wikitext-103-raw-v1", split="train")

# Zugriffsmöglichkeiten auf das DataSet

In [14]:
dataset

Dataset({
    features: ['text'],
    num_rows: 1801350
})

In [15]:
dataset[30000]

{'text': " Le souper de Beaucaire was a political pamphlet written by Napoleon Bonaparte in 1793 . With the French Revolution into its fourth year , civil war had spread across France between various rival political factions . Napoleon was involved in military action , on the government 's side , against some rebellious cities of southern France . It was during these events , in 1793 , that he spoke with four merchants from the Midi and heard their views . As a loyal soldier of the Republic he responded in turn , set on dispelling the fears of the merchants and discouraging their beliefs . He later wrote about his conversation in the form of a pamphlet , calling for an end to the civil war . \n"}

In [16]:
dataset[30000:30005]

{'text': [" Le souper de Beaucaire was a political pamphlet written by Napoleon Bonaparte in 1793 . With the French Revolution into its fourth year , civil war had spread across France between various rival political factions . Napoleon was involved in military action , on the government 's side , against some rebellious cities of southern France . It was during these events , in 1793 , that he spoke with four merchants from the Midi and heard their views . As a loyal soldier of the Republic he responded in turn , set on dispelling the fears of the merchants and discouraging their beliefs . He later wrote about his conversation in the form of a pamphlet , calling for an end to the civil war . \n",
  '',
  ' = = Background = = \n',
  '',
  ' During the French Revolution the National Convention became the executive power of France , following the execution of King Louis XVI . With powerful members , such as Maximilien Robespierre and Georges Danton , the Jacobin Club , a French political

# DataSet für den Trainnigsprozess bereitstellen (Corpus)
## A) als Liste von Listen
Dies lässt sich einfach umsetzen, hat aber den Nachteil, dass die Daten im Hauptspeicher gehalten werden müssen.

In [17]:
batch_size = 1000
all_texts = [dataset[i : i + batch_size]["text"] for i in range(0, len(dataset), batch_size)]

## B) Als Iterator
Ein (Python-) Iterator ist eine Möglichkeit, Daten sukzessive nach und nach zu laden. Da das DataSet die Daten auf der Festplatte vorhält, wird für den Iterator immer nur ein Teil der Daten in den Hauptspeicher geladen.

Wir definieren diesen hier als Funktion:

In [18]:
def batch_iterator():
    for i in range(0, len(dataset), batch_size):
        yield dataset[i : i + batch_size]["text"]

# Verschiedene Tokenizer aufbauen
# 1) Die Architektur eines vorhandenen Tokenizers clonen aber selbst trainieren
[Credits to: https://github.com/huggingface/notebooks/blob/main/examples/tokenizer_training.ipynb]

In [19]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")

# Wir stellen sicher, dass wir eine "fast" Version der Architektur geladen haben, um das Training im Rahmen zu halten
if not tokenizer.is_fast:
    raise ("Dieser Tokenizer ist nicht geeignet.")
else:
    print("Dieser Tokenizer ist geeignet.")

Dieser Tokenizer ist geeignet.


In [20]:
gpt_clone_tokenizer = tokenizer.train_new_from_iterator(batch_iterator(), vocab_size=25000)

In [21]:
outp(gpt_clone_tokenizer(dataset[30000:30005]["text"]))


input_ids
[799, 14477, 559, 421, 1138, 11367, 6875, 321, 259, 2051, 22798, 1753, 369, 9402, 20243, 284, 18437, 272, 1756, 261, 1535, 5176, 649, 529, 2381, 606, 264, 2991, 1018, 455, 4617, 1999, 2328, 771, 2047, 5051, 2051, 14665, 272, 9402, 321, 2559, 284, 1902, 2275, 264, 325, 261, 1385, 329, 83, 1615, 264, 901, 840, 23028, 4536, 282, 2620, 2328, 272, 621, 321, 732, 1128, 2512, 264, 284, 18437, 264, 360, 354, 7103, 357, 833, 13714, 402, 261, 2653, 73, 289, 5001, 531, 5925, 272, 739, 259, 7259, 9361, 282, 261, 2648, 354, 5981, 284, 1324, 264, 915, 325, 2743, 2083, 261, 12922, 282, 261, 13714, 289, 1106, 11149, 286, 531, 11155, 272, 538, 772, 1388, 759, 395, 11610, 284, 261, 839, 282, 259, 22798, 264, 4409, 338, 386, 797, 290, 261, 2991, 1018, 272, 316]
[]
[301, 301, 4789, 301, 301, 316]
[]
[1346, 261, 1535, 5176, 261, 1509, 9764, 920, 261, 4626, 1421, 282, 2328, 264, 1209, 261, 9407, 282, 1279, 3127, 15015, 41, 272, 1756, 4310, 1529, 264, 841, 346, 13531, 3242, 274, 2986, 1302, 945, 2

### Diesen (trainierten!) Tokenizer können wir auch zur Wiederverwertung speichern:

In [22]:
gpt_clone_tokenizer.save_pretrained("../data/gpt_clone_tokenizer")

('../data/gpt_clone_tokenizer\\tokenizer_config.json',
 '../data/gpt_clone_tokenizer\\special_tokens_map.json',
 '../data/gpt_clone_tokenizer\\vocab.json',
 '../data/gpt_clone_tokenizer\\merges.txt',
 '../data/gpt_clone_tokenizer\\added_tokens.json',
 '../data/gpt_clone_tokenizer\\tokenizer.json')

## 2) Architektur eines Tokenizers selbst aufbauen
### Beispiel: Byte Pair Encodings (wie für GPT-2)

In [23]:
# Benötigte Imports
from tokenizers import decoders, models, normalizers, pre_tokenizers, processors, trainers, Tokenizer

## Tokenizer-Pipeline
"Um zu verstehen, wie man einen Tokenizer von Grund auf erstellt, müssen wir ein wenig mehr in die Tokenizer-Bibliothek und die Tokenisierungs-Pipeline eintauchen. Diese Pipeline besteht aus mehreren Schritten:

* **Normalisierung**: Führt alle anfänglichen Transformationen über die anfängliche Eingabezeichenkette aus. Wenn Sie z.B. einen Text klein schreiben, ihn vielleicht entfernen oder einen der üblichen Unicode-Normalisierungsprozesse anwenden wollen, fügen Sie einen Normalizer hinzu.
* **Pre-Tokenizer**: Verantwortlich für die Aufteilung der ursprünglichen Eingabezeichenfolge. Das ist die Komponente, die entscheidet, wo und wie die ursprüngliche Zeichenkette vorsegmentiert wird. Das einfachste Beispiel wäre, einfach an Leerzeichen zu trennen.
* **Modell**: Übernimmt die gesamte Erkennung und Generierung von Sub-Token. Dieser Teil kann trainiert werden und ist wirklich von den Eingabedaten abhängig.
* **Post-Processing**: Bietet erweiterte Konstruktionsfunktionen, um mit einigen der Transformers-basierten SoTA-Modelle kompatibel zu sein. Für BERT wird der tokenisierte Satz beispielsweise um [CLS]- und [SEP]-Token "verpackt"."

Vgl. https://github.com/huggingface/notebooks/blob/main/examples/tokenizer_training.ipynb

### Ein BPE-Tokenizer benötigt einen Pre-Tokenizer (word based tokens)

In [24]:
# Basismodell ist ein BPE-Tokenizer
tokenizer = Tokenizer(models.BPE())

In [25]:
print (tokenizer.pre_tokenizer)
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)

None


In [26]:
# BPE Byte Level Pre-Tokenizer werden über huggingface zur Verfügung gestellt.
tokenizer.pre_tokenizer.pre_tokenize_str("Tokenizer sind toll!")
# add_prefix_space=False verhindert, dass vor dem ersten Wort ebenfalls das Trennzeichen 'Ġ' erscheint. Alle weiteren Token werden mit diesem Zeichen eingeleitet.
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)


In [27]:
# Ergebnisse des Pre-Tokenizers anziegen
tokenizer.pre_tokenizer.pre_tokenize_str("Tokenizer sind toll!")

[('Tokenizer', (0, 9)),
 ('Ġsind', (9, 14)),
 ('Ġtoll', (14, 19)),
 ('!', (19, 20))]

### Unseren Tokenizer trainieren
Der Tokenizer muss trainiert werden, denn er muss die Byte Pairs lernen.

In [28]:
trainer = trainers.BpeTrainer(vocab_size=25000, special_tokens=["<|endoftext|>"])
tokenizer.train_from_iterator(batch_iterator(), trainer=trainer)

### Post-Prozessing
Der Tokenizer erhält noch ein Post-Prozessing.
Der ByteLevel PostProcessor kümmert sich um das Trimming der Offsets. Whitespaces können optional ebenfalls getrimmt werden.


In [29]:
tokenizer.post_processor = processors.ByteLevel(trim_offsets=False)
tokenizer.decoder = decoders.ByteLevel()

### Tokenizer für Huggingface Framework "verpacken"
HuggingFace stellt Wrapper zur Verfügung, die sicherstellen, dass die Tokenizer zu den APIs der Modelle passen.

Hier verwenden wir einen GPT2TokenizerFast.


In [30]:
from transformers import GPT2TokenizerFast

custom_gpt_tokenizer = GPT2TokenizerFast(tokenizer_object=tokenizer)

### Neuen Tokenizer anwenden

In [31]:
outp(custom_gpt_tokenizer(dataset[30000:30005]["text"]))


input_ids
[747, 14425, 507, 369, 1086, 11315, 6823, 269, 207, 1999, 22746, 1701, 317, 9350, 20191, 232, 18385, 220, 1704, 209, 1483, 5124, 597, 477, 2329, 554, 212, 2939, 966, 403, 4565, 1947, 2276, 719, 1995, 4999, 1999, 14613, 220, 9350, 269, 2507, 232, 1850, 2223, 212, 273, 209, 1333, 277, 83, 1563, 212, 849, 788, 22976, 4484, 230, 2568, 2276, 220, 569, 269, 680, 1076, 2460, 212, 232, 18385, 212, 308, 302, 7051, 305, 781, 13662, 350, 209, 2601, 73, 237, 4949, 479, 5873, 220, 687, 207, 7207, 9309, 230, 209, 2596, 302, 5929, 232, 1272, 212, 863, 273, 2691, 2031, 209, 12870, 230, 209, 13662, 237, 1054, 11097, 234, 479, 11103, 220, 486, 720, 1336, 707, 343, 11558, 232, 209, 787, 230, 207, 22746, 212, 4357, 286, 334, 745, 238, 209, 2939, 966, 220, 264]
[]
[249, 249, 4737, 249, 249, 264]
[]
[1294, 209, 1483, 5124, 209, 1457, 9712, 868, 209, 4574, 1369, 230, 2276, 212, 1157, 209, 9355, 230, 1227, 3075, 14963, 41, 220, 1704, 4258, 1477, 212, 789, 294, 13479, 3190, 222, 2934, 1250, 893, 213